In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/03 00:31:07 WARN Utils: Your hostname, codespaces-7c27fa resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
24/03/03 00:31:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/03 00:31:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.version

'3.5.1'

In [ ]:
# !wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

In [ ]:
!wc -l fhvhv_tripdata_2021-01.csv

In [ ]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.show()

In [ ]:
df.schema

In [ ]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [ ]:
!head -n 10 head.csv

In [ ]:
%pip install -U pandas==2.1.3

In [ ]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items # needed to work with spark dataframes with spark version 3.3.2

In [ ]:
df_pandas = pd.read_csv('head.csv')

In [ ]:
df_pandas.dtypes

In [ ]:
spark.createDataFrame(df_pandas).schema

In [ ]:
from pyspark.sql import types

In [ ]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.show()

In [ ]:
df.head(10)

In [ ]:
df = df.repartition(24) # partition the data into 24 partitions, will only be applied when dataframe action
                        # happens like writing to parquet files

In [ ]:
df.write.parquet('fhvhv/2021/01/')

In [ ]:
df = spark.read.parquet('fhvhv/2021/01/')

In [ ]:
df

In [ ]:
df.printSchema()

In [ ]:
from pyspark.sql import functions as F

In [ ]:
df.show()

In [ ]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [ ]:
crazy_stuff('B02884')

In [ ]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [ ]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

In [ ]:
df.select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')

In [ ]:
!head -n 10 head.csv

In [ ]:
pd.DataFrame.items = pd.DataFrame.iteritems # revert this back

In [ ]:
# git test

In [4]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f73ff42bd60>>